In [1]:
# system
import os
import osgeo
# del os.environ['PROJ_LIB']


# import sys, time, ogr
# import libpysal

# basics
import csv
import numpy as np
import math
# from collections import Counter
# from datetime import datetime, timedelta
import geopandas as gpd
# import fiona
# from fulcrum import Fulcrum
# geo
import pandas as pd
from geopandas.tools import sjoin
# from pyproj import proj
from shapely.geometry import Point, shape, box
from osgeo import gdal, gdalconst, osr
# plot 
# import matplotlib.pyplot as plt

# raster

import rasterio
# from rasterstats import zonal_stats
# import xarray as xr



gdal.UseExceptions()  # not required, but a good idea
gdal.AllRegister()



In [12]:
def process_NDVI_image(src_img, dst_img, nir_band_id=4, red_band_id=1):
    with rasterio.open(src_img) as src:

        tiles = src.block_windows(red_band_id)
        out_profile = src.profile

        out_meta = src.meta
        out_meta.update({
             "nodata": -9})
        
        out_profile.update(
            dtype=rasterio.float32,
            count=1,
            compress='lzw',
            nodatavals=-9,
            nodata=-9,
            BIGTIFF = "IF_SAFER")

        with rasterio.open(dst_img, 'w', **out_profile) as dst:
            for idx, window in tiles:
                # print("Processing Block: ", idx[0]+1, ", ", idx[1]+1)
                np.seterr(divide='ignore', invalid='ignore')
                
                red_data = src.read(red_band_id, window=window)

                nir_data = src.read(nir_band_id, window=window)

                # ndvi_con=np.where(
                #     (nir_data.astype(float)+red_data.astype(float))==0., #It will return 0 for No Data value
                #     0,
                #     (nir_data.astype(float) - red_data.astype(float)) / (nir_data.astype(float) + red_data.astype(float)))
                ndvi_con = (nir_data.astype(float) - red_data.astype(float)) / (nir_data.astype(float) + red_data.astype(float))

               
                # L = 0.5
                # savi_con=np.where(
                #     (nir_data.astype(float)+red_data.astype(float))==0., #It will return 0 for No Data value
                #     0,
                #     (nir_data.astype(float) - red_data.astype(float)) / (nir_data.astype(float) + red_data.astype(float) + L) * (1 + L))
                # savi_con =(nir_data.astype(float) - red_data.astype(float)) / (nir_data.astype(float) + red_data.astype(float) + L) * (1 + L)
                # stacked_bands = np.stack((ndvi_con), axis=0)
                # dst.write(stacked_bands, window=window)
                dst.write_band(1, ndvi_con, window=window)
    return


# this creates a binary raster beaased ont he threshold
def process_NDVI_BIN(src_img, dst_img, band_id=1, threshold=0.3):
    with rasterio.open(src_img) as src:

        tiles = src.block_windows(band_id)
        out_profile = src.profile
        ndval = src.nodatavals[0]


        out_profile.update(
            dtype=rasterio.int16,
            count=1,
            nodata=-9,
            nodatavals=-9,
            compress='lzw')
        
        with rasterio.open(dst_img, 'w', **out_profile) as dst:
            for idx, window in tiles:
                # print("Processing Block: ", idx[0]+1, ", ", idx[1]+1)
                dst_data = src.read(band_id, window=window)
                
                dst_data[dst_data==ndval] = np.nan
                dst_data[dst_data<threshold]=  0
                dst_data[dst_data>=threshold] = 1

                
                dst_data[np.isnan(dst_data)]= -9
                dst.write_band(band_id, dst_data, window=window)
    return

In [13]:
import os
directory = r"D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO_000160930_PNEO-geoserve\6band"
for filename in os.listdir(directory):
    if filename.endswith('.tif'):
        with open(os.path.join(directory, filename)) as src:
            
            # run NDVI function from A-Z
            infile =os.path.join(directory, filename)
            dstNDVI = infile.replace('6band','NDVI')
            dstNDVI = dstNDVI.replace('.tif','-NDVI.tif')
            dstNDVIBIN = dstNDVI.replace('-NDVI.tif','-NDVI-BIN-03.tif')
            print(infile)
            # process_NDVI_image(infile, dstNDVI)#, nir_band_id=4, red_band_id=1)
            process_NDVI_BIN(dstNDVI, dstNDVIBIN, band_id=1, threshold=0.3)

            

D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO_000160930_PNEO-geoserve\6band\DIM_PNEO3_202301200801484_PMS-FS_ORT_PWOI_000160930_6_1_F_1.tif
-9.0
D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO_000160930_PNEO-geoserve\6band\DIM_PNEO3_202301200802044_PMS-FS_ORT_PWOI_000160930_5_1_F_1.tif
-9.0
D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO_000160930_PNEO-geoserve\6band\DIM_PNEO3_202301200802330_PMS-FS_ORT_PWOI_000160930_9_1_F_1.tif
-9.0
D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO_000160930_PNEO-geoserve\6band\DIM_PNEO3_202301300754465_PMS-FS_ORT_PWOI_000160930_7_1_F_1.tif
-9.0
D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO_000160930_PNEO-geoserve\6band\DIM_PNEO3_202303030809452_PMS-FS_ORT_PWOI_000160930_16_1_F_1.tif
-9.0
D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO_000160930_PNEO-geoserve\6band\DIM_PNEO4_202302020802268_PMS-FS_ORT_PWOI_000160930_10_1_F_1.tif
-9.0
D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO_000160930_PNEO-geoserve\6band\DIM_PNEO4_202302180809076_PMS-FS_ORT_PWOI_000

In [2]:
res=[]
def bboxraster(src_img):
    with rasterio.open(src_img) as src:

        rabounds  = src.bounds

        arr = src.read(1)

        uni = np.unique(arr[arr>=0], return_counts=True)
        pixlarea=abs(src.profile['transform'][0]*src.profile['transform'][4])

        nonveg= uni[1][0]*pixlarea
        veg = uni[1][1]*pixlarea
        relveg =veg/(nonveg+veg)*100
        print(uni) 

        res.append({'img':infile,'nonveg': nonveg, 'veg': veg,'relveg':relveg,'geom': box(*rabounds) })

    return(res)

In [3]:
directory = r"D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO_000160930_PNEO-geoserve\ndvi"
for filename in os.listdir(directory):
    if filename.endswith('NDVI-BIN.tif'):
        with open(os.path.join(directory, filename)) as src:
            
            # run NDVI function from A-Z
            infile =os.path.join(directory, filename)

            print(infile)
            bboxraster(infile)#, nir_band_id=4, red_band_id=1)

D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO_000160930_PNEO-geoserve\ndvi\DIM_PNEO3_202301200801484_PMS-FS_ORT_PWOI_000160930_6_1_F_1-NDVI-BIN.tif
(array([0, 1], dtype=int16), array([141122205,  45027989], dtype=int64))
D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO_000160930_PNEO-geoserve\ndvi\DIM_PNEO3_202301200802044_PMS-FS_ORT_PWOI_000160930_5_1_F_1-NDVI-BIN.tif
(array([0, 1], dtype=int16), array([2824355314, 2517779301], dtype=int64))
D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO_000160930_PNEO-geoserve\ndvi\DIM_PNEO3_202301200802330_PMS-FS_ORT_PWOI_000160930_9_1_F_1-NDVI-BIN.tif
(array([0, 1], dtype=int16), array([4842576836, 2494771935], dtype=int64))
D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO_000160930_PNEO-geoserve\ndvi\DIM_PNEO3_202301300754465_PMS-FS_ORT_PWOI_000160930_7_1_F_1-NDVI-BIN.tif
(array([0, 1], dtype=int16), array([996128534, 968567581], dtype=int64))
D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO_000160930_PNEO-geoserve\ndvi\DIM_PNEO3_202303030809452_PMS-FS

In [4]:
pd.DataFrame(res)
df = gpd.GeoDataFrame(pd.DataFrame(res),geometry='geom')

In [10]:
df.veg.sum()/(df.veg.sum()+df.nonveg.sum())*100

18.96572938120098

In [12]:
df.relveg.mean()

16.59118224707888

In [13]:
df

,img,nonveg,veg,relveg,geom
0,D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO...,1.269965e+07,4.052090e+06,24.189064,"POLYGON ((618778.500 2384957.100, 618778.500 2..."
1,D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO...,2.541849e+08,2.265938e+08,47.130585,"POLYGON ((620337.300 2342055.900, 620337.300 2..."
2,D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO...,4.358203e+08,2.245235e+08,34.001000,"POLYGON ((608688.900 2340602.700, 608688.900 2..."
3,D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO...,8.964799e+07,8.716760e+07,49.298595,"POLYGON ((627801.300 2359199.100, 627801.300 2..."
4,D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO...,3.861624e+07,7.886541e+05,2.001412,"POLYGON ((583421.700 2384450.700, 583421.700 2..."
5,D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO...,1.909508e+07,2.155960e+07,53.031031,"POLYGON ((627884.100 2352805.500, 627884.100 2..."
6,D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO...,2.087465e+08,7.391097e+06,3.419625,"POLYGON ((584838.900 2371249.500, 584838.900 2..."
7,D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO...,5.005509e+08,3.033205e+07,5.713510,"POLYGON ((572855.700 2339501.100, 572855.700 2..."
8,D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO...,1.022662e+07,5.216392e+05,4.853245,"POLYGON ((565395.300 2384756.700, 565395.300 2..."
9,D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO...,1.368892e+08,8.396855e+06,5.779535,"POLYGON ((560332.500 2347434.300, 560332.500 2..."


In [ ]:
df.nan/df.veg*100

In [14]:
df.to_file(r"D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO_000160930_PNEO-geoserve\ndvi\output-03.gpkg", driver="GPKG")

In [15]:
df.to_csv(r"D:\BPLA Dropbox\08 GIS-data\GIS-data-Mekkah\WO_000160930_PNEO-geoserve\ndvi\output-03.csv")